In [1]:
import os
import h5py
import numpy as np
from scipy import misc
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import tensorflow as tf
'''
from tensorflow.python.keras.models import Input, Sequential, Model
from tensorflow.python.keras.layers import Dense, Dropout, LSTM, Activation, Add
from tensorflow.python.keras.utils import Sequence
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.optimizers import adam, SGD
from tensorflow.python.keras.callbacks import ReduceLROnPlateau
from tensorflow.python.keras.utils import plot_model
'''

'\nfrom tensorflow.python.keras.models import Input, Sequential, Model\nfrom tensorflow.python.keras.layers import Dense, Dropout, LSTM, Activation, Add\nfrom tensorflow.python.keras.utils import Sequence\nfrom tensorflow.python.keras import backend as K\nfrom tensorflow.python.keras.optimizers import adam, SGD\nfrom tensorflow.python.keras.callbacks import ReduceLROnPlateau\nfrom tensorflow.python.keras.utils import plot_model\n'

In [3]:
from modis_utils.misc import cache_data, restore_data

In [4]:
import keras
from keras.models import Input, Sequential, Model
from keras.layers import Dense, Dropout, LSTM, Activation, Add
from keras.utils import Sequence
from keras import backend as K
from keras.optimizers import adam, SGD
from keras.callbacks import ReduceLROnPlateau
from keras.utils import plot_model

Using TensorFlow backend.


In [5]:
config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )
sess = tf.Session(config=config)

In [6]:
data = restore_data('cache/boundary_vectors_ALL.h5')

In [7]:
train_boundary_vectors = data[0]
val_boundary_vectors = data[1]
test_boundary_vectors = data[2]

In [8]:
n_points = train_boundary_vectors.shape[1]
n_points

1024

In [9]:
train_boundary_vectors.shape, val_boundary_vectors.shape, test_boundary_vectors.shape

((438, 1024, 2), (138, 1024, 2), (138, 1024, 2))

In [10]:
def transform(data, scaler, flatten=True):
    old_shape = data.shape
    data = data.reshape(old_shape[0], -1)
    data = scaler.transform(data.astype(np.float))
    if not flatten:
        return data.reshape(old_shape)
    return data

In [11]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train_boundary_vectors.reshape(train_boundary_vectors.shape[0], -1))
train_boundary_vectors_scale = transform(train_boundary_vectors, scaler, flatten=True)
val_boundary_vectors_scale = transform(val_boundary_vectors, scaler, flatten=True)
test_boundary_vectors_scale = transform(test_boundary_vectors, scaler, flatten=True)

In [12]:
# normalize the dataset
train_boundary_vectors_scale_1 = transform(train_boundary_vectors, scaler, flatten=False)
val_boundary_vectors_scale_1 = transform(val_boundary_vectors, scaler, flatten=False)
test_boundary_vectors_scale_1 = transform(test_boundary_vectors, scaler, flatten=False)

In [13]:
def create_dataset(boundary_vectors_scale, timesteps):
    data_X = []
    data_Y = []
    for i in range(len(boundary_vectors_scale) - timesteps):
        data_x = boundary_vectors_scale[i:(i+timesteps)]
        data_y = boundary_vectors_scale[i + timesteps]
        data_X.append(data_x)
        data_Y.append(data_y)
    return np.asarray(data_X), np.asarray(data_Y)

In [14]:
timesteps = 50
train_X, train_Y = create_dataset(train_boundary_vectors_scale, timesteps)
val_X, val_Y = create_dataset(np.concatenate(
    [train_boundary_vectors_scale[-timesteps:], val_boundary_vectors_scale]),
                              timesteps)
test_X, test_Y = create_dataset(np.concatenate(
    [val_boundary_vectors_scale[-timesteps:], test_boundary_vectors_scale]),
                                timesteps)

In [15]:
timesteps = 50
train_X_1, train_Y_1 = create_dataset(train_boundary_vectors_scale_1, timesteps)
val_X_1, val_Y_1 = create_dataset(np.concatenate(
    [train_boundary_vectors_scale_1[-timesteps:], val_boundary_vectors_scale_1]),
                              timesteps)
test_X_1, test_Y_1 = create_dataset(np.concatenate(
    [val_boundary_vectors_scale_1[-timesteps:], test_boundary_vectors_scale_1]),
                                timesteps)

In [16]:
train_X.shape, train_Y.shape, val_X.shape, val_Y.shape, test_X.shape, test_Y.shape

((388, 50, 2048),
 (388, 2048),
 (138, 50, 2048),
 (138, 2048),
 (138, 50, 2048),
 (138, 2048))

In [17]:
train_X_1.shape, train_Y_1.shape, val_X_1.shape, val_Y_1.shape, test_X_1.shape, test_Y_1.shape

((388, 50, 1024, 2),
 (388, 1024, 2),
 (138, 50, 1024, 2),
 (138, 1024, 2),
 (138, 50, 1024, 2),
 (138, 1024, 2))

In [18]:
def plot_model_(model):
    plot_model(model, to_file='model.png', show_shapes=True)
    model_plot = misc.imread('model.png')
    plt.imshow(model_plot)

In [19]:
def create_graph_matrix(n_points_on_boundary):
    def calc_arc_distance(a, b, n):
        diff = np.abs(a-b)
        if diff > n//2:
            diff = n - diff
        return diff
    
    n = n_points_on_boundary
    mat = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            mat[i,j] = calc_arc_distance(i, j, n)
    return mat.astype(np.float32)

def create_graph_matrix_1(n_points_on_boundary):
    def calc_arc_distance(a, b, n):
        diff = np.abs(a-b)
        if diff > n//2:
            diff = n - diff
        return diff
    
    n = n_points_on_boundary
    mat = np.zeros((2*n, 2*n))
    for i in range(n):
        for j in range(n):
            mat[i,j] = calc_arc_distance(i, j, n)
    mat[n:2*n, n:2*n] = mat[:n, :n]
    for i in range(n):
        for j in range(n, 2*n):
            mat[i,j] = mat[i, j - n]
    mat[n:2*n, :n] = mat[:n, n:2*n]
    return mat.astype(np.float32)

In [20]:
#a = create_graph_matrix_1(1024)
#a_1 = create_graph_matrix(n_points)
#a.shape, a_1.shape

In [21]:
from keras_dgl.layers import GraphConvLSTM, GraphConvLSTM_1

In [22]:
def create_model(timesteps, n_points, mat=None):
    K.clear_session()
    
    if mat is None:
        mat = create_graph_matrix(n_points)
    if len(mat.shape) == 2:
        mat = np.expand_dims(mat, axis=0)
    
    model = Sequential()
    model.add(GraphConvLSTM_1(4, graph_conv_tensor=mat, return_sequences=True, activation='tanh',
                              input_shape=(timesteps, n_points, 2)))
    model.add(GraphConvLSTM_1(4, graph_conv_tensor=mat, return_sequences=True, activation='tanh'))
    model.add(GraphConvLSTM_1(2, graph_conv_tensor=mat, return_sequences=False, activation='sigmoid'))
    
    model.compile(loss='mse', optimizer=adam(lr=0.001))
    return model

In [23]:
from time import time

In [24]:
start = time()
model = create_model(timesteps, 12)
end = time()
print(end - start)

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


UnboundLocalError: local variable 'f' referenced before assignment

In [ ]:
model.summary()

In [ ]:
model.weights

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.00001, verbose=1)

In [ ]:
history = model.fit(train_X, train_Y, epochs=20, validation_data=(val_X, val_Y),
                    verbose=1, callbacks=[reduce_lr])